# CSS 120

## Climate Change Models

### Umberto Mignozzetti (UCSD)

(Based on Project Pythia and ClimateMatch)

## Today's lecture

In this lecture, we will keep studying the physics of climate change.

1. Climate feedbacks
1. Multiplicity of equilibrium
1. Changing in forcings (insolation)
1. Adding dimensionality (vertical models)
1. Earth System Models

## Packages

In [ ]:
!pip install metpy --quiet

## Packages

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import holoviews as hv
import panel as pn
from scipy.optimize import brentq
import xarray as xr
import climlab
import pooch
import os
import tempfile

# New package!
import metpy                     # used to make Skew T Plots of temperature and pressure
from metpy.plots import SkewT    # plotting function used widely in climate science

## Packages

In [ ]:
def pooch_load(filelocation=None, filename=None, processor=None):
    shared_location = "~/"
    user_temp_cache = tempfile.gettempdir()
    if os.path.exists(os.path.join(shared_location, filename)):
        file = os.path.join(shared_location, filename)
    else:
        file = pooch.retrieve(
            filelocation,
            known_hash=None,
            fname=os.path.join(user_temp_cache, filename),
            processor=processor,
        )
    return file

#  make animations stuff
plt.rcParams["animation.html"] = "jshtml"

def initial_figure(model):
    with plt.ioff():  # will hide the inital figure which will plot separate from the video otherwise
        fig = plt.figure(figsize=(6, 6))
        lines = []

        skew = SkewT(fig, rotation=30)
        #  plot the observations
        skew.plot(
            Tglobal.level,
            Tglobal,
            color="black",
            linestyle="-",
            linewidth=2,
            label="Observations",
        )
        lines.append(
            skew.plot(
                model.lev,
                model.Tatm - climlab.constants.tempCtoK,
                linestyle="-",
                linewidth=2,
                color="C0",
                label="RC model (all gases)",
            )[0]
        )
        skew.ax.legend()
        skew.ax.set_ylim(1050, 10)
        skew.ax.set_xlim(-60, 75)
        # Add the relevant special lines
        skew.plot_dry_adiabats(linewidth=1.5, label="dry adiabats")
        # skew.plot_moist_adiabats(linewidth=1.5, label = 'moist adiabats')
        skew.ax.set_xlabel("Temperature ($^\circ$C)", fontsize=14)
        skew.ax.set_ylabel("Pressure (hPa)", fontsize=14)
        lines.append(
            skew.plot(
                1000,
                model.Ts - climlab.constants.tempCtoK,
                "o",
                markersize=8,
                color="C0",
            )[0]
        )

    return fig, lines


def animate(day, model, lines):
    lines[0].set_xdata(np.array(model.Tatm) - climlab.constants.tempCtoK)
    lines[1].set_xdata(np.array(model.Ts) - climlab.constants.tempCtoK)
    # lines[2].set_xdata(np.array(model.q)*1E3)
    # lines[-1].set_text('Day {}'.format(int(model.time['days_elapsed'])))
    # This is kind of a hack, but without it the initial frame doesn't appear
    if day != 0:
        model.step_forward()
    return lines


# to setup the skewT and plot observations
def make_basic_skewT():
    fig = plt.figure(figsize=(9, 9))
    skew = SkewT(fig, rotation=30)
    skew.plot(
        Tglobal.level,
        Tglobal,
        color="black",
        linestyle="-",
        linewidth=2,
        label="Observations",
    )
    skew.ax.set_ylim(1050, 10)
    skew.ax.set_xlim(-90, 45)
    # Add the relevant special lines
    # skew.plot_dry_adiabats(linewidth=1.5, label = 'dry adiabats')
    # skew.plot_moist_adiabats(linewidth=1.5, label = 'moist adiabats')
    # skew.plot_mixing_lines()
    skew.ax.legend()
    skew.ax.set_xlabel("Temperature (degC)", fontsize=14)
    skew.ax.set_ylabel("Pressure (hPa)", fontsize=14)
    return skew


# to setup the skewT and plot observations
def make_skewT():
    fig = plt.figure(figsize=(9, 9))
    skew = SkewT(fig, rotation=30)
    skew.plot(
        Tglobal.level,
        Tglobal,
        color="black",
        linestyle="-",
        linewidth=2,
        label="Observations",
    )
    skew.ax.set_ylim(1050, 10)
    skew.ax.set_xlim(-90, 45)
    # Add the relevant special lines
    skew.plot_dry_adiabats(linewidth=1.5, label="dry adiabats")
    # skew.plot_moist_adiabats(linewidth=1.5, label = 'moist adiabats')
    # skew.plot_mixing_lines()
    skew.ax.legend()
    skew.ax.set_xlabel("Temperature (degC)", fontsize=14)
    skew.ax.set_ylabel("Pressure (hPa)", fontsize=14)
    return skew


# to add a model derived profile to the skewT figure
def add_profile(skew, model, linestyle="-", color=None):
    line = skew.plot(
        model.lev,
        model.Tatm - climlab.constants.tempCtoK,
        label=model.name,
        linewidth=2,
    )[0]
    skew.plot(
        1000,
        model.Ts - climlab.constants.tempCtoK,
        "o",
        markersize=8,
        color=line.get_color(),
    )
    skew.ax.legend()

# Climate Feedbacks

## Ice-Albedo Feedback: Temperature Dependent Albedo

Our current model only contains one feedback, the 'Planck feedback' also called the 'Planck temperature response'. 

This feedback encapsulates that a warming of Earth leads to the planet emitting more energy. 

In reality, there are many  [climate feedbacks](https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_AnnexVII.pdf) that contribute to the Earth's net temperature change due to an energy imbalance. 

In this section, we will focus on incorporating an **ice-albedo feedback** into our model.

## Ice-Albedo Feedback: Temperature Dependent Albedo

When the Earth's surface warms, snow and ice melt.

This lowers the **albedo (**$\mathbf{\alpha}$**)**, because less solar radiation is reflected off Earth's surface. 

This lower albedo causes the climate to warm even more than if the albedo had stayed the same, increasing the snow and ice melt. 

> This is referred to as a **positive feedback**.

Positive feedbacks amplify the changes that are already occurring. 

This particular feedback is referred to as the **ice-albedo feedback**.

## Ice-Albedo Feedback: Temperature Dependent Albedo

A simple way to parameterize ice-albedo feedback in our model is through a temperature-dependent albedo, such as the one defined below:

\begin{align}
\alpha = \left\{
        \begin{array}{cl}
        0.1 & T \gt 300 \text{ K} \\
        0.1 + (0.7-0.1) \cdot \frac{(T-300)^2}{(240-300)^2} & 240\text{ K} \le T \le 300\text{ K} \\
        0.7 & T \lt 240\text{ K}
        \end{array}
        \right.
\end{align}

Using this new temperature-dependent albedo, we can plot the graphs of absorbed shortwave radiation $\left(ASR\right)$ and outgoing longwave radiation $\left(OLR\right)$

## Ice-Albedo Feedback: Temperature Dependent Albedo

In [ ]:
# create a array ot temperatures to evaluates the ASR and OLR at
T = np.arange(200, 360, 2, dtype=np.float64)

# create empty arrays to fill with values later
ASR_vals = np.zeros_like(T)

# define the slope of the ramp function
m = (0.7 - 0.3) / (280 - 250)

# define the observed insolation based on observations from the IPCC AR6 Figure 7.2
Q = 340  # W m^-2

# define transmissivity
tau = 0.6127  # unitless number between 0 and 1

## Ice-Albedo Feedback: Temperature Dependent Albedo

In [ ]:
# define a function for absorbed shortwave radiation (ASR)
def ASR(Q, T):
    # define function for albedo
    if T >= 300:  # temperature of very warm and ice free earth.
        alpha = 0.1  # average albedo of land and sea without ice
    elif T > 240:  # temperature of Earth to sustain permafrost and sea ice everywhere.
        alpha = 0.1 + (0.7 - 0.1) * (T - 300) ** 2 / (240 - 300) ** 2
    else:
        alpha = 0.7  # average albedo of land and sea ice
    return (1 - alpha) * Q

## Ice-Albedo Feedback: Temperature Dependent Albedo

In [ ]:
# define a function for outgoing longwave radiation (OLR)
def OLR(tau, T):
    # define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
    sigma = 5.67e-8  # W m^-2 K^-4

    return tau * sigma * T**4


# calculate OLR for different values of T
OLR_vals = OLR(tau, T)

# calculate ASR for different values of T
for tt, temp in enumerate(T):
    ASR_vals[tt] = ASR(Q, temp)

## Ice-Albedo Feedback: Temperature Dependent Albedo

In [ ]:
# make plots
fig, ax = plt.subplots()
ax.plot(T, ASR_vals, label="Absorbed Shortwave Radiation ($ASR$)")
ax.plot(T, OLR_vals, label="Outgoing Longwave Radiation ($OLR$)")


ax.set_xlabel("Temperature (K)")
ax.set_ylabel("Radiative Flux (Wm$^{-2}$)")
ax.legend()

## Multiplicity of Equilibria

## Multiple Equilibria

**Equilibrium temperatures** are solutions to the model equation when the rate of change of temperature is zero. 

There are two types of equilibrium solutions: *stable* and *unstable*.

- A *stable equilibrium* temperature is a solution that the model asymptotes to (moves towards) over time. 
- An *unstable equilibrium* temperature is a solution that the model diverges (moves away) from over time. The only time the model will stay at this equilibrium is if it starts *exactly* at the unstable equilibrium temperature. 
 
We can now incorporate the temperature-dependent albedo we defined above into our time-dependent model.

## Multiple Equilibria

In [ ]:
# create a function to find the new temperature based on the previous using Euler's method.
def step_forward(T, tau, Q, dt):

    # define the heat capacity
    C = 286471954.64  # J m^-2K

    T_new = T + dt / C * (ASR(Q, T) - OLR(tau, T))

    return T_new

## Multiple Equilibria

Let us explore how our model behaves under a variety of initial temperatures.

We can use a `for` loop to compare different initial temperatures.

In [ ]:
dt = 60.0 * 60.0 * 24.0 * 365.0  # time interval, one year expressed in seconds

fig, ax = plt.subplots()
for init_temp in T:  # suite of initial temperatures in K
    numtsteps = 40  #  number of years to run the model
    # for converting the number of seconds in a year
    sec_2_yr = 3.154e7
    # set the initial temperature (initial condition)
    T_series = [init_temp]
    # set the initial time to 0
    t_series = [0]
    # run the model
    for n in range(numtsteps):
        # calculate and append the time since running the model, dependent on dt and the numtsteps
        t_series.append((n + 1) * dt / sec_2_yr)
        # calculate and append the new temperature using our pre-defined function
        T_series.append(step_forward(T_series[n], tau=tau, Q=Q, dt=dt))
    # make plot
    ax.plot(t_series, T_series)

ax.set_title("Temporal Evolution of Temperature"); ax.set_xlabel("Time (years)"); ax.set_ylabel("Temperature (K)")

## Finding Equilibria Numerically & Determining Convergence or Divergence

To verify the equilibrium solutions we identified graphically in the previous section, we can use python to find the exact values where the rate of change in temperature is zero:

\begin{align}
0 = ASR-OLR.
\end{align}

To aid us, we will use [brentq](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html#scipy-optimize-brentq), a *root-finding function* from the `scipy` package.

## Finding Equilibria Numerically & Determining Convergence or Divergence

In [ ]:
# create function to find the forcing at the top of the atmosphere
def Ftoa(T):
    return ASR(Q, T) - OLR(tau, T)


#  brentq() requires a function and two end-points to be input as arguments
#  it will look for a root/zero of the function between those end-points
Teq1 = brentq(
    Ftoa, 200.0, 240.0
)  # these ranges are from the intersections of the graphs of ASR and OLR
Teq2 = brentq(Ftoa, 240.0, 280.0)
Teq3 = brentq(Ftoa, 280.0, 320.0)

print(Teq1, Teq2, Teq3)

## Finding Equilibria Numerically & Determining Convergence or Divergence

To assess the stability of these equilibria, we can plot the difference in $ASR$ and $OSR$. 

This is the same function `Ftoa` that we calculated in the previous cell, but we will recalculate it below for plotting purposes.

In [ ]:
# we've already calculated ASR and OLR above
fig, ax = plt.subplots()
F = ASR_vals - OLR_vals
ax.plot(T, F, color="k", linewidth=3)

# find positive values and fill with red
pos_ind1 = T <= Teq1
ax.fill_between(T[pos_ind1], 0, F[pos_ind1], color="red")

pos_ind2 = (T >= Teq2) & (T <= Teq3)
ax.fill_between(T[pos_ind2], 0, F[pos_ind2], color="red")

# find negative values and fill with blue
neg_ind1 = (T >= Teq1) & (T <= Teq2)
ax.fill_between(T[neg_ind1], 0, F[neg_ind1], color="blue")

neg_ind2 = T >= Teq3
ax.fill_between(T[neg_ind2], 0, F[neg_ind2], color="blue")

# plot vertical lines/names at equilibrium temperatures
ax.axvline(x=Teq1, color="k", ls=":")
ax.axvline(x=Teq2, color="k", ls=":")
ax.axvline(x=Teq3, color="k", ls=":")

ax.annotate(
    "$T_{eq1}$",
    xy=(Teq1 - 5, -295),
    xytext=(Teq1 - 5, -295),
    rotation=90,
    annotation_clip=False,
)
ax.annotate(
    "$T_{eq2}$",
    xy=(Teq2 - 5, -295),
    xytext=(Teq2 - 5, -295),
    rotation=90,
    annotation_clip=False,
)
ax.annotate(
    "$T_{eq3}$",
    xy=(Teq3 - 5, -295),
    xytext=(Teq3 - 5, -295),
    rotation=90,
    annotation_clip=False,
)

# plot arrows/text to show stability of equilibrium points
ax.annotate(
    "",
    xy=(232, -50),
    xytext=(200, -50),
    arrowprops=dict(facecolor="black", arrowstyle="-|>"),
)

ax.annotate(
    "",
    xy=(242.5, -50),
    xytext=(233, -50),
    arrowprops=dict(facecolor="black", arrowstyle="<|-"),
)

ax.annotate(
    "",
    xy=(305.5, -50),
    xytext=(243.5, -50),
    arrowprops=dict(facecolor="black", arrowstyle="-|>"),
)

ax.annotate(
    "",
    xy=(358, -50),
    xytext=(307, -50),
    arrowprops=dict(facecolor="black", arrowstyle="<|-"),
)


ax.annotate("convergence", xy=(358, -170), xytext=(307, -170), rotation=90)

ax.annotate("divergence", xy=(305.5, -170), xytext=(243.5, -170), rotation=90)

ax.annotate("convergence", xy=(242.5, -170), xytext=(233, -170), rotation=90)


ax.set_xlabel("Temperature (K)")
ax.set_ylabel("$ASR-OLR$ (Wm$^{-2}$)");

## Finding Equilibria Numerically & Determining Convergence or Divergence

The red regions represent conditions where the Earth would warm, because the energy absorbed by the Earth system is greater than the energy emitted or reflected back into space. 

The blue regions represent conditions where the Earth would cool, because the outgoing radiation is larger than the absorbed radiation.

For example, if Earth started at an initial temperature below $T_{\text{eq1}}$ (in the left red region), it will move to the right on the $x$-axis, towards the $T_{\text{eq1}}$ equilibrium state.

Conversely, if Earth started between $T_{\text{eq1}}$ and $T_{\text{eq1}}$ (the left blue region), the temperature would decrease, moving left on the $x$-axis until it reaches $T_{\text{eq1}}$.

Thus $T_{\text{eq1}}$ is a *stable* equilibrium as the temperature curves will tend to this point after a long time.

# Changing Insolation

## Changing Insolation

Insolation fluctuates with time.

Over Earth's history, the insolation has sometimes been lower, and sometimes higher, than the currently observed $340 \text{ Wm}^{-2}$.

These insolation changes directly affect the $ASR$, causing Earth to warm or cool depending on whether it receives more or less insolation respectively.

To look at the effect that changing insolation has on Earth's equilibrium state(s), we can re-plot $ASR$ as a function of temperature for several different insolation values, alongside the $OLR$.

## Changing Insolation

In [ ]:
# define the observed insolation
Q_vals = [220, 340, 420]  # W m^-2

fig, ax = plt.subplots()
for Q_2 in Q_vals:
    # calculate ASR and OLR for different values of T
    for tt, temp in enumerate(T):

        ASR_vals[tt] = ASR(Q_2, temp)

    # make plots
    ax.plot(T, ASR_vals, label="$ASR$ for $Q$ = " + str(Q_2) + " Wm$^{-2}$")

# note we calculated OLR previously, and it does not depend on Q
ax.plot(T, OLR_vals, label="$OLR$"); ax.set_xlabel("Temperature (K)")
ax.set_ylabel("Radiative Flux (Wm$^{-2}$)"); ax.legend()

## Changing Insolation

As we increase or decrease the insolation, the number of intersections between $ASR$ and $OLR$ can change!

This means the number of equilibrium solutions for our model will also change.

### Effect on Equilibrium Temperatures

To understand how this effect translates to different equilibrium temperatures of our model over time, we will apply a range of insolation values to our model.

Let us first start off with a very cold Earth, at $220 \text{K}$, and warm the Earth by steadily increasing the insolation above our present-day $340 \text{ Wm}^{-2}$ value.

## Changing Insolation

In [ ]:
# these are the values of insolation we will use
insolation_vals = np.arange(340, 500, 3)

# initial temperature we will use
init_temp = 220  # K

fig, ax = plt.subplots()

for i, insolation in enumerate(insolation_vals):  # suite of initial temperatures in K

    numtsteps = 100  #  number of years to run the model

    # for converting the number of seconds in a year
    sec_2_yr = 3.154e7

    # set the initial temperature (initial condition)
    T_series = [init_temp]

    # set the initial time to 0
    t_series = [0]

    # run the model
    for n in range(numtsteps):

        # calculate and append the time since running the model, dependent on dt and the numtsteps
        t_series.append((n + 1) * dt / sec_2_yr)

        # calculate and append the new temperature using our pre-defined function
        T_series.append(step_forward(T_series[n], tau=tau, Q=insolation, dt=dt))

    # make plot
    colors = plt.cm.coolwarm(np.linspace(0, 1, insolation_vals.shape[0]))
    if (
        insolation == 385
    ):  # This is just to highlight a particularly interesting insolation value
        ax.plot(t_series, T_series, color=colors[i], linestyle="dashed")
    else:
        ax.plot(t_series, T_series, color=colors[i])

ax.set_ylabel("Temperature (K)")
ax.set_xlabel("Time (years)")

# Vertical Structure of the Atmosphere

## Vertical Structure of the Atmosphere

The energy balance model we have been using is _zero-dimensional_, yielding only the global mean surface temperature.

We might ask, is it possible to construct a similar, _one-dimensional_, model for an _atmospheric column_ to estimate the global mean temperature _profile_ (i.e., including the height/$z$ dimension)?

Additionally, can we explicitly include the effects of different gases in this model, rather than just parametrizing their collective effects through a single parameter $\tau$? 

> **The answer is yes, we can!**

## Vertical Structure of the Atmosphere

Since it is too complex to construct from scratch, we will use a model already available within the python package [`climlab`](https://climlab.readthedocs.io/en/latest/intro.html).

The model we will first use is a radiative equilibrium model. 

> **Radiative equilibrium models** consider different layers of the atmosphere. Each of these layers absorbs and emits radiation depending on its constituent gases, allowing the model to calculate the radiation budget for each layer as radiative energy is transferred between atmospheric layers, the Earth's surface, and space. 

**Radiative equilibrium** is reached when each layer gains energy at the same rate as it loses energy.

## Vertical Structure of the Atmosphere

To set up this model, we will need information about some of the mean properties of the atmosphere.

We are going to download water vapor data from the Community Earth System Model (CESM), to use a variable called [specific humidity](https://glossary.ametsoc.org/wiki/Specific_humidity).

**Specific humidity** is the mass of water vapor per mass of a unit block of air. This is useful because water vapor is an important greenhouse gas.

## Vertical Structure of the Atmosphere

In [ ]:
filename_sq = "cpl_1850_f19-Q-gw-only.cam.h0.nc"
url_sq = "https://osf.io/c6q4j/download/"

ds = xr.open_dataset(
    pooch_load(filelocation=url_sq, filename=filename_sq)
)  # ds = dataset
ds

## Vertical Structure of the Atmosphere

The specific humidity is stored in a variable called `Q`:

In [ ]:
ds.Q

## Vertical Structure of the Atmosphere

In [ ]:
ds.time

## Vertical Structure of the Atmosphere

However, we want an annual average profile.

We need to take global, annual average using a weighting (ds.gw) that is calculated based on the model grid - and is similar, but not identical, to a cosine(latitude) weighting:

In [ ]:
weight_factor = ds.gw / ds.gw.mean(dim="lat")
Qglobal = (ds.Q * weight_factor).mean(dim=("lat", "lon", "time"))
Qglobal

## Vertical Structure of the Atmosphere

Now that we have a global mean water vapor profile, we can define a model that has the same vertical levels as this water vapor data.

In [ ]:
# use 'lev=Qglobal.lev' to create an identical vertical grid to water vapor data
mystate = climlab.column_state(lev=Qglobal.lev, water_depth=2.5)
mystate

## Vertical Structure of the Atmosphere

To model the absorption and emission of different gases within each atmospheric layer, we use the **[Rapid Radiative Transfer Model](https://climlab.readthedocs.io/en/latest/api/climlab.radiation.RRTMG.html)**, which is contained within the `RRTMG` module.

We must first initialize our model using the water vapor.

In [ ]:
radmodel = climlab.radiation.RRTMG(
    name="Radiation (all gases)",      # give our model a name
    state=mystate,                     # give our model an initial condition
    specific_humidity=Qglobal.values,  # tell the model how much water vapor there is
    albedo=0.25,                       # set the SURFACE shortwave albedo
    # set the timestep to one day (measured in seconds)
    timestep=climlab.constants.seconds_per_day,
)
radmodel

## Vertical Structure of the Atmosphere

Let's explore this initial state. Here `Ts` is the initial global mean surface temperature, and `Tatm` is the initial global mean air temperature profile.

In [ ]:
radmodel.state

## Vertical Structure of the Atmosphere

One of the perks of using this model is its ability to incorporate the radiative effects of individual greenhouse gases in different parts of the radiation spectrum, rather than using a bulk reduction in transmission of outgoing longwave radiation (as in our previous models).

Let's display `absorber_vmr`, which contains the **volume mixing ratio**'s of each gas used in the radiative transfer model (these are pre-defined; and do not include the water vapor we used as a model input above).

The volume mixing ratio describes the fraction of molecules in the air that are a given gas.

For example, $21\%$ of air is oxygen so its volume mixing ratio is $0.21$.

## Vertical Structure of the Atmosphere

In [ ]:
radmodel.absorber_vmr

## Vertical Structure of the Atmosphere

To look at carbon dioxide (`"CO2"`) in a more familiar unit, parts per million (by volume), we can convert and print the new value.

In [ ]:
radmodel.absorber_vmr["CO2"] * 1e6

## Vertical Structure of the Atmosphere

We can also look at all the available diagnostics of our model:

In [ ]:
diag_ds = climlab.to_xarray(radmodel.diagnostics)
diag_ds

## Vertical Structure of the Atmosphere

For example to look at $OLR$ (Note that the $OLR$ is currently 0 as we have not run the model forward in time, so it has not calculated any radiation components):

In [ ]:
radmodel.OLR

# Vertical Structure of the Atmosphere: Compare Data to the Model

## Vertical Structure of the Atmosphere: Compare Data to the Model

Before we run our model forward, we will download a reanalysis product from NCEP to get a sense of what the real global mean atmospheric temperature profile looks like.

We will compare this profile to our model runs later.

In [ ]:
filename_ncep_air = "air.mon.1981-2010.ltm.nc"
url_ncep_air = "https://osf.io/w6cd5/download/"
ncep_air = xr.open_dataset(
    pooch_load(filelocation=url_ncep_air, filename=filename_ncep_air)
)  # ds = dataset

# this is the long term monthly means (note only 12 time steps)
ncep_air.air

## Vertical Structure of the Atmosphere: Compare Data to the Model

In [ ]:
# need to take the average over space and time
# the grid cells are not the same size moving towards the poles, so we weight by the cosine of latitude to compensate for this
coslat = np.cos(np.deg2rad(ncep_air.lat))
weight = coslat / coslat.mean(dim="lat")

Tglobal = (ncep_air.air * weight).mean(dim=("lat", "lon", "time"))
Tglobal

## Vertical Structure of the Atmosphere: Compare Data to the Model

Below we will define two helper funcitons to visualize the profiles output from our model with a _SkewT_ plot.

This is common way to plot atmospheric temperature in climate science, and the `metpy` package has a built in function to make this easier.

In [ ]:
# to setup the skewT and plot observations
def make_skewT():
    fig = plt.figure(
        figsize=(9, 9)
    )
    skew = SkewT(fig, rotation=30)
    skew.plot(
        Tglobal.level,
        Tglobal,
        color="black",
        linestyle="-",
        linewidth=2,
        label="Observations",
    )
    skew.ax.set_ylim(1050, 10)
    skew.ax.set_xlim(-90, 45)
    # Add the relevant special lines
    # skew.plot_dry_adiabats(linewidth=1.5, label = 'dry adiabats')
    # skew.plot_moist_adiabats(linewidth=1.5, label = 'moist adiabats')
    # skew.plot_mixing_lines()
    skew.ax.legend()
    skew.ax.set_xlabel("Temperature (degC)"
                       #, fontsize=14
                      )
    skew.ax.set_ylabel("Pressure (hPa)"
                       #, fontsize=14
                      )
    return skew

## Vertical Structure of the Atmosphere: Compare Data to the Model

In [ ]:
# to add a model derived profile to the skewT figure
def add_profile(skew, model, linestyle="-", color=None):
    line = skew.plot(
        model.lev,
        model.Tatm - climlab.constants.tempCtoK,
        label=model.name,
        linewidth=2,
    )[0]
    skew.plot(
        1000,
        model.Ts - climlab.constants.tempCtoK,
        "o",
        markersize=8,
        color=line.get_color(),
    )
    skew.ax.legend()

## Vertical Structure of the Atmosphere: Compare Data to the Model

SkewT (also known as SkewT-logP) plots are generally used for much [more complex reasons](https://www.weather.gov/source/zhu/ZHU_Training_Page/convective_parameters/skewt/skewtinfo.html) than we will use here.

However, one of the benefits of this plot that we will utilize is the fact that pressure decreases approximately logarithmically with height.

Thus, with a _logP_ axis, we are showing information that is roughly linear in height, making the plots more intuitive.

In [ ]:
skew = make_skewT()

# Earth System Models

## Earth System Model data

ESMs include the physical processes typical of General Circulation Models (GCMs), but also include chemical and biological changes within the climate system (e.g. changes in vegetation, biomes, atmospheric CO$_2$).

The several systems simulated in an ESM (ocean, atmosphere, cryosphere, land) are coupled to each other, and each system has its own variables, physics, and discretizations -- both of the spatial grid and the timestep.

<img src="https://upload.wikimedia.org/wikipedia/commons/7/73/AtmosphericModelSchematic.png" alt= “EarthSystemModel” width="420" height="400">

Atmospheric Model Schematic (Credit: [Wikipedia](https://upload.wikimedia.org/wikipedia/commons/7/73/AtmosphericModelSchematic.png))

## Questions?

## See you in the next class!